In [1]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from scipy.special import expit, logit

In [37]:
def sigmoid(x):
    #return (1.0/(1.0+np.exp(-x)))
    return expit(x)
def tahn(x):
    return np.tanh(x)    
def relu(x):
    return np.maximum(0,x)
def linear(x):
    return x
def deriv_sigmoid(x):
    #return sigmoid(x)*(1-sigmoid(x))
    return expit(x)*(1-expit(x))
def deriv_tahn(x):
    return -4/(np.exp(x)-np.exp(-x))**2
def deriv_relu(x):
    return (x>0)*1
def deriv_linear(x):
    return 1
def forward_propagation(x_row,layers, activation_functions):
    for layer in layers:
        x_row = np.dot(layer["weights"],x_row.T)+layer["b_coefs"]
        for i in range(len(x_row)):
            x_row[i]=activation_functions[layer["activation"]](x_row[i])
        layer["output"]=x_row
def backward_propagation(layers, y_train, deriv_functions):
    errors = []
    for j in range(len(layers[-1]["output"])):
        errors.append(y_train[j]-layers[-1]["output"][j])
    errors = np.array(errors)
    deriv = []
    for i in range(len(layers[-1]['output'])):
        deriv.append(deriv_functions[layers[-1]["activation"]](layers[-1]['output'][i]))
    adamar = [errors[j]*deriv[j] for j in range(len(errors))]
    layers[-1]["err"]=adamar
    for i in reversed(range(len(layers)-1)):
        error = np.dot(layers[i+1]["err"], layers[i+1]["weights"])
        for j in range(len(layers[i]['output'])):
            deriv.append(deriv_functions[layers[i]["activation"]](layers[i]['output'][j]))
        adamar = [error[j]*deriv[j] for j in range(len(error))]
        layers[i]["err"]=adamar
def update_weights(x_row, layers, learning_rate):
    DWl = learning_rate*np.dot(np.array([x_row]).T,np.array([layers[0]["err"]]))
    layers[0]["weights"]+=DWl.T
    DBl = np.dot(learning_rate,layers[0]["err"])
    layers[0]["b_coefs"]+=DBl.T
    for i in range(1,len(layers)):
        inputs = np.array([layers[i-1]["output"]])
        DWl = learning_rate*np.dot(inputs.T,np.array([layers[i]["err"]]))
        layers[i]["weights"]+=DWl.T
        DBl = np.dot(learning_rate,layers[i]["err"])
        layers[i]["b_coefs"]+=DBl.T

In [38]:
class Perceptron:
    def __init__(self, input_layer_size, hidden_layers):
        self.activation_functions = dict(sigmoid=sigmoid,tahn=tahn,relu=relu, linear=linear)
        self.deriv_functions = dict(sigmoid=deriv_sigmoid,tahn=deriv_tahn,relu=deriv_relu, linear=deriv_linear)
        self.layers = []
        w = np.random.rand (hidden_layers[0][0], input_layer_size)
        b = np.random.rand (hidden_layers[0][0])
        self.layers.append(dict(weights=w,b_coefs=b, activation=hidden_layers[0][1]))
        for i in range(1, len(hidden_layers)):
            w = np.random.rand (hidden_layers[i][0], hidden_layers[i-1][0])
            b = np.random.rand (hidden_layers[i][0])
            self.layers.append(dict(weights=w,b_coefs=b,activation=hidden_layers[i][1]))
    def fit(self, X_train, y_train, learn_rate, epohs):
        for ep in range(epohs):
            for ind in range((len(X_train))):
                forward_propagation(np.array(X_train[ind]), self.layers, self.activation_functions)
                backward_propagation(self.layers, y_train[ind], self.deriv_functions)
                update_weights(X_train[ind], self.layers, learn_rate)
    def predict(self, X_test):
        predictions = []
        for row in X_test:
            forward_propagation(row, self.layers, self.activation_functions)
            predictions.append(self.layers[-1]["output"])
        return predictions

In [39]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression().fit(X_train, y_train)
lr.coef_

array([[16.81464604, 54.20951909,  5.18005235, 63.5083138 , 93.6190598 ,
        70.58882409, 86.96159313, 10.4337126 ,  3.29343253, 70.8523127 ]])

In [40]:
from sklearn.datasets import make_regression
X, y = make_regression(n_samples=100, n_features=10, noise=1, random_state=42)
#obj = MinMaxScaler() 
y = y.reshape(-1, 1)
#X= obj.fit_transform(x)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4)
print(y_train)

[[ -38.89658826]
 [ 138.76049731]
 [ -60.65958208]
 [ -58.8906285 ]
 [  16.92677163]
 [ 146.78496357]
 [  64.61898396]
 [ 132.92680268]
 [-355.66423507]
 [-109.93389215]
 [-159.58094264]
 [  38.88223786]
 [-186.39069357]
 [ 160.21539642]
 [ 191.30025868]
 [-148.35469239]
 [-354.58646521]
 [ -46.67504919]
 [ -98.71664701]
 [ 262.6559801 ]
 [ 377.19809646]
 [-281.19763115]
 [   0.52152413]
 [ 161.39291439]
 [  -9.87089318]
 [  39.22973697]
 [-406.36252019]
 [-365.50150894]
 [ 211.89573217]
 [ 197.64298327]
 [ 190.39562803]
 [  39.39882104]
 [  73.22456819]
 [-103.74951521]
 [  16.03994359]
 [ 366.74882215]
 [  76.77339076]
 [  85.48801481]
 [ 465.68844512]
 [ 179.53139266]
 [ -60.26144965]
 [-293.87786497]
 [  17.1293582 ]
 [ -77.52083248]
 [ -98.61248973]
 [ 238.53269152]
 [ 373.12354607]
 [  15.36864403]
 [ 289.26620865]
 [-158.15033612]
 [   3.56714038]
 [ 416.77316588]
 [ 255.66531859]
 [-360.5475471 ]
 [ 127.99956075]
 [-145.23488949]
 [ -38.15125076]
 [-123.47711619]
 [  21.8229572

In [78]:
per = Perceptron(10, [[64,"sigmoid"],[32,"sigmoid"],[1,"linear"]])

In [79]:
per.fit(X_train, y_train, 0.005, 10)

In [80]:
pred = per.predict(X_test)

In [81]:
print(pred)

[array([-112.50587018]), array([120.67458904]), array([120.67458904]), array([-112.50587018]), array([-112.50587018]), array([120.67458904]), array([-112.50587018]), array([120.67458904]), array([120.67458904]), array([-112.50587018]), array([120.67458904]), array([120.67458904]), array([120.67458904]), array([-112.50587018]), array([120.67458904]), array([-112.50587018]), array([120.67458904]), array([120.67458904]), array([120.67458904]), array([-112.50587018]), array([-112.50587018]), array([-112.50587018]), array([120.67458904]), array([120.67458904]), array([120.67458904]), array([120.67458904]), array([-112.50587018]), array([-112.50587018]), array([120.67458904]), array([120.67458904]), array([120.67458904]), array([-112.50587018]), array([-112.50587018]), array([-112.50587018]), array([120.67458904]), array([-112.50587018]), array([-112.50587018]), array([-112.50587018]), array([-112.50587018]), array([-112.50587018])]


In [82]:
print(y_test)

[[-261.183329  ]
 [  51.28847653]
 [ 186.72080998]
 [-162.76957725]
 [  12.19785968]
 [-101.579094  ]
 [-184.20688467]
 [  26.00922358]
 [ 229.33371172]
 [ -60.83052466]
 [ 199.99362731]
 [ 110.84399422]
 [  63.12718183]
 [ -48.35491808]
 [  76.43264171]
 [-251.49474496]
 [ 128.98672403]
 [ 256.02764328]
 [  68.30429409]
 [-112.67436388]
 [-198.53063462]
 [ -63.42363463]
 [ 250.52332157]
 [ -48.87139956]
 [ 154.23079591]
 [  73.34829567]
 [-127.54306781]
 [ -59.14680874]
 [ 294.33012726]
 [ 134.82107885]
 [ 120.87376973]
 [  -3.38782441]
 [-157.44201739]
 [-500.76289929]
 [ 208.05859858]
 [-202.7240423 ]
 [ -12.27333509]
 [ -95.34531404]
 [-180.22336428]
 [-118.09637393]]


In [56]:
per.layers[0]

{'weights': array([[ 5.69736020e+08,  1.03336219e+09,  3.06823829e+08,
          5.86337322e+08,  1.39748237e+09,  1.73716039e+09,
          2.18897977e+09, -7.55664201e+07,  2.47096983e+08,
          1.52256417e+09],
        [ 1.21630240e+08,  2.40972742e+08,  6.96305015e+07,
          1.44722035e+08,  3.27965245e+08,  4.38987716e+08,
          5.63985600e+08, -1.95496283e+07,  5.04722146e+07,
          3.65885494e+08],
        [ 6.22868077e+07,  1.04159514e+08,  3.19871856e+07,
          5.26915257e+07,  1.37491525e+08,  1.51371983e+08,
          1.83628043e+08, -1.09112139e+07,  2.72748092e+07,
          1.45125943e+08],
        [ 5.39400026e+07,  9.04492089e+07,  2.77849034e+07,
          4.57634693e+07,  1.19319482e+08,  1.31335629e+08,
          1.59382889e+08, -9.57137340e+06,  2.36987976e+07,
          1.25964300e+08],
        [ 5.39448199e+07,  9.04567990e+07,  2.77870967e+07,
          4.57671680e+07,  1.19329513e+08,  1.31345993e+08,
          1.59395420e+08, -9.57207860e+06

In [12]:
def initil(input_layer_size, hidden_layers):
        layers = []
        w = np.random.rand (hidden_layers[0][0], input_layer_size)
        b = np.random.rand (hidden_layers[0][0])
        layers.append(dict(weights=w,b_coefs=b, activation=hidden_layers[0][1]))
        for i in range(1, len(hidden_layers)):
            w = np.random.rand (hidden_layers[i][0], hidden_layers[i-1][0])
            b = np.random.rand (hidden_layers[i][0])
            layers.append(dict(weights=w,b_coefs=b,activation=hidden_layers[i][1]))
        return layers

In [92]:
test = initil(4, [[2,"sigmoid"],[1,"sigmoid"]])
print(test[0])

{'weights': array([[0.95744274, 0.60391834, 0.0011658 , 0.57279748],
       [0.0285874 , 0.6078318 , 0.21409916, 0.19511582]]), 'b_coefs': array([0.65981875, 0.51201985]), 'activation': 'sigmoid'}


In [93]:
activation_functions = dict(sigmoid=sigmoid,tahn=tahn,relu=relu)
forward_propagation(np.array([1,2,3,4]), test, activation_functions)
test[1]

[0.99405821 0.96003753]
[0.73517155]


{'weights': array([[0.13472736, 0.5026855 ]]),
 'b_coefs': array([0.40449784]),
 'activation': 'sigmoid',
 'output': array([0.73517155])}

In [94]:
a=np.array([1,2,3,4])
print(test[2])

IndexError: list index out of range

In [95]:
deriv_functions =  dict(sigmoid=deriv_sigmoid,tahn=deriv_tahn,relu=deriv_relu)
backward_propagation(test, [1], deriv_functions)
print(test[0])

{'weights': array([[0.95744274, 0.60391834, 0.0011658 , 0.57279748],
       [0.0285874 , 0.6078318 , 0.21409916, 0.19511582]]), 'b_coefs': array([0.65981875, 0.51201985]), 'activation': 'sigmoid', 'output': array([0.99405821, 0.96003753]), 'err': [0.001540827386400851, 0.005838323141099926]}


In [96]:
print(test[1])

{'weights': array([[0.13472736, 0.5026855 ]]), 'b_coefs': array([0.40449784]), 'activation': 'sigmoid', 'output': array([0.73517155]), 'err': [0.058009461740434316]}


In [97]:
update_weights(np.array([1,2,3,4]), test, 1)

In [98]:
test[0]

{'weights': array([[ 0.95590191,  0.60083669, -0.00345668,  0.56663417],
        [ 0.02274908,  0.59615515,  0.19658419,  0.17176253]]),
 'b_coefs': array([0.65827792, 0.50618152]),
 'activation': 'sigmoid',
 'output': array([0.99405821, 0.96003753]),
 'err': [0.001540827386400851, 0.005838323141099926]}

In [99]:
test[1]

{'weights': array([[0.07706258, 0.44699423]]),
 'b_coefs': array([0.34648838]),
 'activation': 'sigmoid',
 'output': array([0.73517155]),
 'err': [0.058009461740434316]}

In [105]:
test[2]

IndexError: list index out of range

In [106]:
forward_propagation(np.array([1,2,3,4]), test, activation_functions) 

Поступило [1 2 3 4]
Веса [[0.44819388 0.31703907 0.38669645 0.77813263]
 [0.26956988 0.03063332 0.95025344 0.82998768]]
После преобраз [[5.99904494 6.98416215]]
После активации [[0.99752502 0.99907442]]
Поступило [[0.99752502 0.99907442]]
Веса [[0.54461951 0.39151847]]
После преобраз [[1.92713187]]
После активации [[0.87293162]]


In [107]:
test[0]

{'weights': array([[0.44819388, 0.31703907, 0.38669645, 0.77813263],
        [0.26956988, 0.03063332, 0.95025344, 0.82998768]]),
 'b_coefs': array([[0.64415305, 0.48261457]]),
 'activation': 'sigmoid',
 'output': array([[0.99752502, 0.99907442]]),
 'err': [array([0.00290189, 0.00203921])]}

In [108]:
test[1]

{'weights': array([[0.54461951, 0.39151847]]),
 'b_coefs': array([[0.99270419]]),
 'activation': 'sigmoid',
 'output': array([[0.87293162]]),
 'err': array([[0.02856033]])}